In [17]:
from convml_tt.system import TripletTrainerModel, TripletTrainerDataModule

import torch

In [2]:
arch = "resnet18"
#model = TripletTrainerModel(base_arch=arch)
model = TripletTrainerModel.load_from_checkpoint("/home/earlcd/Downloads/epoch=19-step=2819.ckpt")

In [3]:
datamodule = TripletTrainerDataModule(
    data_dir="../data/Nx256_s200000.0_N0study_N100train/",
    normalize_for_arch=arch,
    batch_size=32
)
datamodule.setup(stage="fit")

In [4]:
train_dl = datamodule.train_dataloader()
len(train_dl)
batch = list(train_dl)[0]

In [5]:
model.freeze()
model.summarize()


  | Name     | Type       | Params
----------------------------------------
0 | backbone | Sequential | 11.2 M
1 | head     | Sequential | 533 K 
----------------------------------------
0         Trainable params
11.7 M    Non-trainable params
11.7 M    Total params
46.838    Total estimated model params size (MB)


  | Name     | Type       | Params
----------------------------------------
0 | backbone | Sequential | 11.2 M
1 | head     | Sequential | 533 K 
----------------------------------------
0         Trainable params
11.7 M    Non-trainable params
11.7 M    Total params
46.838    Total estimated model params size (MB)

In [6]:
batch[0].shape

torch.Size([32, 3, 256, 256])

In [26]:
model.training_step(batch, 0)

tensor(1., grad_fn=<MeanBackward0>)

In [7]:
ys = [model.encoder(x) for x in batch]

In [9]:
y_anchor, y_neighbour, y_distant = ys

In [10]:
l_near = ((y_anchor - y_neighbour) ** 2.0).sum(dim=1)
l_near

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])

In [12]:
l_distant = ((y_anchor - y_distant) ** 2.0).sum(dim=1)
l_distant

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])

In [18]:
torch.mean(l_near**0.3 + torch.exp(-l_distant))

tensor(1.)

In [20]:
1.0/l_near**0.3

tensor([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf])

In [15]:
[p for p in model.parameters()]

[Parameter containing:
 tensor([[[[-1.2398e-17, -4.8283e-17,  2.4489e-17,  ..., -2.2343e-18,
            -1.1114e-17, -2.2844e-17],
           [ 4.8452e-19, -5.1620e-19, -1.1533e-17,  ..., -5.7202e-17,
            -1.1824e-16, -3.7788e-16],
           [-1.6190e-19, -3.5453e-19, -1.1118e-18,  ..., -9.7575e-18,
            -2.5245e-16, -1.7477e-16],
           ...,
           [-5.9923e-19, -1.1229e-18, -1.2764e-19,  ..., -2.9035e-19,
            -6.6880e-18, -9.4325e-17],
           [ 1.2119e-17, -7.3391e-18, -1.2817e-18,  ..., -5.8168e-20,
            -2.5800e-18, -4.2085e-17],
           [-8.4905e-17, -5.2366e-17, -1.7941e-18,  ..., -1.9866e-19,
            -1.3959e-18, -2.6886e-17]],
 
          [[-4.7718e-16, -5.6601e-16, -2.2937e-16,  ..., -2.3585e-17,
            -1.2019e-15, -3.7144e-16],
           [-2.1634e-17, -5.2717e-17, -3.2277e-16,  ..., -9.0920e-16,
            -1.6273e-15, -3.8250e-16],
           [-7.1252e-18, -2.7750e-17, -9.9651e-17,  ..., -2.7415e-16,
            -6.8